Audio Data should be processed before fed into a model. The processing can be done either by converting the audio into spectrum images (that can be fed to Convolutional Models) or the numeric equivalents of the spectrum (that can be fed to High Ending Machine Learning Model).

_thinkdsp_ is a very good package for working with Digital Signal Processing. I will be using this here.

In [0]:
!git clone https://github.com/AllenDowney/ThinkDSP.git

Cloning into 'ThinkDSP'...
remote: Enumerating objects: 2086, done.
remote: Total 2086 (delta 0), reused 0 (delta 0), pack-reused 2086
Receiving objects: 100% (2086/2086), 190.44 MiB | 29.33 MiB/s, done.
Resolving deltas: 100% (1111/1111), done.


In [0]:
import sys
sys.path.insert(0, '/content/ThinkDSP/code/') 
import thinkdsp

In [0]:
import glob

In [0]:
Globlist = []

I am defining some functions to aid  in extracting the audio data along with labels.

In [0]:
def Cat_Dog_Bird(filename): #This function will return the label based on the filename
    if 'dog_barking' in filename: 
        return 'dog' 
    elif 'Bird' in filename:
        return 'bird'
    else :
        return 'cat'
    

In [0]:

#This function will return the time-position of the frequency that is greater than a 
#certain amount.
def searchInArray(last, freq, data):  
    curr = last
    for s in range(last, data.size):
        if freq < int(data[s]):
            if s > 0:
                 curr = s
                 break
    return curr

In [0]:
#This function will convert the spectrum into corresponding numeric representation.
def ConvSpect(spectrum): 
    pos = 0
    lst = []
    for freq in range(10, 8000, 53):
        pos = searchInArray(pos, freq, spectrum.fs)
        lst.append(int(spectrum.hs[pos].real))
    Globlist.append(lst)

In [0]:
labels =[]
def ReadFile(fileName):
    test_wave = thinkdsp.read_wave(fileName)
    spectrum = test_wave.make_spectrum()
    labels.append(Cat_Dog_Bird(fileName))
    ConvSpect(spectrum)

Now, the filenames are extracted from the directory.

In [0]:
fileBirds = glob.glob("/content/train/bird/*.wav")
fileCats  = glob.glob("/content/train/cat/*.wav")
fileDogs  = glob.glob("/content/train/dog/*.wav")

In [0]:
for animal in [fileBirds, fileCats, fileDogs]:
  for filename in animal:
    ReadFile(filename)

In [42]:
len(Globlist) , len(labels) , len(Globlist) == len(fileBirds) + len(fileDogs) +len(fileCats)

(226, 226, True)

Now, the lists are bound into a DataFrame for proper.

In [52]:
import pandas as pd
df  = pd.DataFrame(Globlist)
df['label'] = labels
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,label
0,7,-20,3,-5,13,3,6,1,0,-2,-1,0,1,0,-2,0,0,2,-2,-2,-3,-1,-1,-2,-2,1,0,-2,0,-7,0,2,-5,-5,-22,11,7,-1,3,-29,...,-17,-3,8,-13,-18,-15,-16,4,7,4,6,4,4,0,1,0,-2,0,0,0,0,0,2,-2,2,0,2,1,0,1,0,2,5,6,-1,3,0,7,3,bird
1,5,-1,-2,0,0,4,2,8,-16,3,5,4,-11,-1,16,14,-7,298,-35,-31,2,15,7,-2,96,20,-10,2,51,72,55,-53,-487,-13,-119,-819,-778,-45,7,-3,...,-3,0,3,3,-8,0,-1,3,0,3,9,-3,-2,0,0,-1,2,-2,0,0,0,0,0,0,2,0,3,2,1,0,-2,-1,0,-3,2,-1,0,-2,-1,bird
2,-52,35,52,-58,-29,-16,-39,18,95,10,-24,18,0,24,78,-40,-8,36,-22,-11,6,37,8,29,-19,-2,11,9,14,-23,-9,-1,-18,0,5,-3,-9,-12,-5,2,...,-30,26,125,-20,-14,-32,24,-2,-15,-6,1,4,-3,0,-1,0,-2,1,0,2,-7,-1,-4,-4,-1,-1,0,0,4,2,0,0,1,-1,2,1,-4,1,4,bird
3,-43,56,31,12,9,13,-12,-4,0,14,20,-9,0,3,-16,-12,4,-25,-5,-13,-13,5,11,8,0,13,-25,-25,38,42,-39,77,118,-51,-110,80,-159,262,-37,-5,...,-14,-11,-8,-6,-3,1,-3,-7,0,5,2,2,0,4,0,0,-5,-2,0,3,5,0,5,0,0,-4,2,6,4,-8,0,21,11,1,-32,0,-7,-23,5,bird
4,0,0,2,-1,-2,-1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,40,-40,15,31,14,103,-5,-19,-3,4,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,bird


In [0]:
df.to_csv("Audionumeric.csv", index=False)